In [ ]:
from google.transit import gtfs_realtime_pb2
import urllib
import urllib.request
import time
import datetime
import os
import multiprocessing
import re
import pandas as pd
from datetime import date
from zipfile import ZipFile
from pyproj import Geod
from google.transit import gtfs_realtime_pb2
import requests
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
from google.protobuf.json_format import MessageToJson
import requests
import json
import pandas as pd
from collections import OrderedDict
import datetime
from protobuf_to_dict import protobuf_to_dict
import mysql.connector
from sqlalchemy import create_engine
from pandas.io import sql
from pyproj import Geod
from gtfs_functions import speed_trip_trajectory_preprocessing_analysis
from datetime import timedelta
from IPython.display import clear_output

In [ ]:
wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method
def monthname(mydate):
    ##Generate the today's month name instead of the selected date
    #mydate = datetime.datetime.now()
    m = mydate.strftime("%B")
    return(m)

def Brisbane(epoch):
    a = time.strftime("%d-%m-%Y %H:%M:%S", time.localtime(epoch))
    return(a)

def createCSVfile(inputlist , name):
    with open( name , 'w') as f:
        for i in inputlist:
            k = 0
            for item in i:  
                f.write(str(item) + ',')
                k = k+1
            f.write('\n')
        return(f)
def inputCSVfile(csvfile):
    list1= []
    with open(csvfile, 'r') as f:
        for i in f:
            j = i.split(',')
            
            le = len(j)
            j[le - 1] = (j[le- 1]).strip()
            list1.append(j)
        return(list1)

def createCSVfileWCD(inputlist , name):
    with open( name , 'w') as f:
        for i in inputlist:
            f.write(str(i))
            f.write('\n')
        return(f)
def Distance(lat1,lon1,lat2,lon2):
    az12 ,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
    return (dist)

In [ ]:
Working_Directory = r"Y:\\Data\\GTFS_NEW\\"
Static_Folder = Working_Directory + os.sep + 'GTFS Static/'
Realtime_folder  = Working_Directory+ os.sep + 'GTFS Realtime/'
Traj_folder = Realtime_folder + os.sep + 'VehicleTrajectory entity/'
if not os.path.exists(Static_Folder):
    os.makedirs(Static_Folder)
if not os.path.exists(Realtime_folder):
    os.makedirs(Realtime_folder)
if not os.path.exists(Traj_folder):
    os.makedirs(Traj_folder)
    
gtfs_static_link = r"https://gtfsrt.api.translink.com.au/GTFS/SEQ_GTFS.zip"
gtfs_realtime_link = r'https://gtfsrt.api.translink.com.au/api/realtime/SEQ/VehiclePositions'

In [ ]:
t1 = 0
datee = datetime.datetime.strptime(str(Brisbane(t1)),"%d-%m-%Y %H:%M:%S" )
m = datee.month
y = datee.year
d = datee.day
m_name = monthname(datee)
date_name1 = str(d)+"-"+str(m)+"-"+str(y)
TableName = str(m_name)+"_"+str(y)
TableName = TableName.lower()
index_o = 0
curr_date = date_name1
########################Between 12:30am to 4:30am Operating Hours#############################
end_time = datetime.time(0,30,0)
start_time = datetime.time(4,30,0)
i = 1
flags = 0

In [ ]:
while True:
    try:
        clear_output(wait=True)
        #Directories for a new Day
        t = int(time.time())
        datee = datetime.datetime.strptime(str(Brisbane(t)),"%d-%m-%Y %H:%M:%S" )
        m = datee.month
        y = datee.year
        d = datee.day
        #m_name = monthname(Brisbane(t))
        m_name = monthname(datee)
        date_name = str(d)+"-"+str(m)+"-"+str(y)
        Month_year = str(m_name)+" ,"+str(y)
        if date_name!= date_name1:
            index_o = 0
            datee = datetime.datetime.strptime(str(Brisbane(t)),"%d-%m-%Y %H:%M:%S" )
            m = datee.month
            y = datee.year
            d = datee.day
            #m_name = monthname(Brisbane(t))
            m_name = monthname(datee)
            date_name = str(d)+"-"+str(m)+"-"+str(y)
            TableName = str(m_name)+"_"+str(y)
            TableName = TableName.lower()
            Month_year = str(m_name)+" ,"+str(y)
            TUdirectory = Realtime_folder + "TripUpdate entity" +"/" + 'TU '+str(Month_year)
            VPdirectory = Realtime_folder + "VehiclePosition entity" +"/" + 'VP '+str(Month_year)
            if not os.path.exists(TUdirectory):
                os.makedirs(TUdirectory)
            TUdirectory2= TUdirectory+ "/"+ 'TU '+str(date_name)
            if not os.path.exists(TUdirectory2):
                os.makedirs(TUdirectory2) 
            TUdirectory3= TUdirectory2+ "/"+ 'TU feeds '+str(date_name)
            TUdirectory4= TUdirectory2+ "/"+ 'TU Speed Analysis'+str(date_name)
            if not os.path.exists(TUdirectory3):
                os.makedirs(TUdirectory3)
            if not os.path.exists(TUdirectory4):
                os.makedirs(TUdirectory4)
            if not os.path.exists(VPdirectory):
                os.makedirs(VPdirectory)
            VPdirectory2= VPdirectory+ "/"+ 'VP '+str(date_name)
            if not os.path.exists(VPdirectory2):
                os.makedirs(VPdirectory2)

            #Downloading and processing static file

            GTFS_Static = urllib.request.urlretrieve(gtfs_static_link, Static_Folder + '/GTFS Static ' +date_name + '.zip')
            zip_file = ZipFile(Static_Folder + '/GTFS Static ' +date_name + '.zip')
            trips = pd.read_csv(zip_file.open('trips.txt'))
            stop_times = pd.read_csv(zip_file.open('stop_times.txt'))
            shapes = pd.read_csv(zip_file.open('shapes.txt'), low_memory=False)
            shapes['shape_pt_sequence'] = shapes['shape_pt_sequence'].astype(str)
            Route_Shape_stop = stop_times.merge(trips, on = 'trip_id', how = 'left')
            Route_Shape_stop = Route_Shape_stop[['shape_id', 'stop_id', 'stop_sequence', 'route_id', 'trip_id']].drop_duplicates(keep = 'first')
            Route_Shape_stop['stop_sequence'] = Route_Shape_stop['stop_sequence'].astype(int)
            Route_Shape_stop['stop_id'] = Route_Shape_stop['stop_id'].astype(str)
            shapes2 = shapes.copy(deep = True)
            shapes2['shape_pt_sequence_next'] = (shapes2['shape_pt_sequence'].astype(int) + 1).astype(str)
            shapes2['stop_seq_1'] = (shapes2['shape_pt_sequence'].astype(int)/10000).astype(int)
            shapes2['stop_seq_2'] = (shapes2['shape_pt_sequence_next'].astype(int)/10000).astype(int)
            shapes3 = shapes2.merge(shapes, left_on = ['shape_id', 'shape_pt_sequence_next'], right_on = ['shape_id', 'shape_pt_sequence'], how = 'left')
            shapes3['distance'] = Distance(shapes3['shape_pt_lat_x'].tolist(),shapes3['shape_pt_lon_x'].tolist(),shapes3['shape_pt_lat_y'].tolist(),shapes3['shape_pt_lon_y'].tolist())
            shapes3['distance'] = shapes3['distance']/1000
            shapes3 = shapes3.loc[shapes3['stop_seq_1'] == shapes3['stop_seq_2']]
            shapes3['stop_seq_2'] = shapes3['stop_seq_2'] + 1
            shapes4 = pd.DataFrame(shapes3.groupby(['shape_id', 'stop_seq_1', 'stop_seq_2'], as_index = False)['distance'].sum())
            distance_file = shapes4
            date_name1 = date_name
                  
        if date_name1 == date_name and curr_date != date_name and end_time <= datee.time() < start_time:
            clear_output(wait=True)
            Trajdirectory = Traj_folder + "Traj " + str(Month_year)
            if not os.path.exists(Trajdirectory):
                os.makedirs(Trajdirectory)
            try:
                timer_st = time.time()
                print("processing date = ", date_name)
                previous_datee = datee + timedelta(-i)
                prev_date_name = str(previous_datee.day)+"-"+str(previous_datee.month)+"-"+str(previous_datee.year)
                prev_m_name = monthname(previous_datee)
                prev_table_name = str(prev_m_name)+"_"+str(previous_datee.year)
                prev_table_name = prev_table_name.lower()
                print(f"processing {prev_date_name}....")
                _, _, df_speed_traj = speed_trip_trajectory_preprocessing_analysis(previous_datee)
                print("processing df_speed trajectory done!")
                print(df_speed_traj.shape)
                timer_end = time.time()
                print(f"The duration it from vp preprocessing = {(timer_end - timer_st)/60} mins")
                prev_table_name = prev_table_name + "_traj"
                df_speed_traj.to_csv(f"{Trajdirectory}/trip_trajectory_{prev_date_name}.csv")
                curr_date = date_name
            except Exception as e:
                print("an exception occurred")
                print(str(e))
#                 break
#                 pass
        #Downloading the feed

        feed = gtfs_realtime_pb2.FeedMessage()
        response = urllib.request.urlopen(gtfs_realtime_link)
        feed.ParseFromString(response.read())
        dict_obj = protobuf_to_dict(feed)
        r = json.dumps(dict_obj)
        loaded_r = json.loads(r)
        if flags ==1:
            kppa = pd.DataFrame(loaded_r['entity'])
            if kppa.shape != kppa1.shape:
                vt = kppa.vehicle.apply(pd.Series)
                vt1 = vt.position.apply(pd.Series)
                vt2 = vt.trip.apply(pd.Series)
                vt4 = vt.vehicle.apply(pd.Series)
                VE = pd.concat([vt, vt1, vt2, vt4], axis=1)
                VE = VE[['current_status', 'stop_id', 'timestamp', 'trip_id','latitude', 'longitude' , 'route_id','id', 'label' ]]
                VE = VE.loc[~VE['timestamp'].isna()]
                VE.to_csv(VPdirectory2 + '/VP feed '+ date_name + ' '+ str(t)  + '.csv')
                index_o = index_o + 1
                kppa1 = kppa.copy(deep = True)
                flags = 1
        if flags == 0:
            kppa = pd.DataFrame(loaded_r['entity'])
            vt = kppa.vehicle.apply(pd.Series)
            vt1 = vt.position.apply(pd.Series)
            vt2 = vt.trip.apply(pd.Series)
            vt4 = vt.vehicle.apply(pd.Series)
            VE = pd.concat([vt, vt1, vt2, vt4], axis=1)
            VE = VE[['current_status', 'stop_id', 'timestamp', 'trip_id','latitude', 'longitude' , 'route_id','id', 'label' ]]
            VE = VE.loc[~VE['timestamp'].isna()]
            VE.to_csv(VPdirectory2 + '/VP feed '+ date_name + ' '+ str(t)  + '.csv')
            index_o = index_o + 1
            kppa1 = kppa.copy(deep = True)
            flags = 1
        response.close()
    except:
        pass